In [1]:
!pip install nltk

In [2]:
import nltk
import pandas as pd
import numpy as np
import ast #why??->reason is below in data preprocessing step
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer

In [3]:
#read the dataset
credits = pd.read_csv("credits.csv")
movies = pd.read_csv("movies.csv")

In [4]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [6]:
#merge the dataset
#since we have merege based on title so title will not com twice but both the dataframe have id column 
# so need to remove from one of them
movie = movies.merge(credits,on="title").drop("movie_id",axis=1)

In [7]:
movie.shape 

(4809, 22)

In [8]:
movie.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [9]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [10]:
#creating the final dataframe 
#removing the unwanted columns
movie = movie[["id","title","keywords","genres", "overview", "cast","crew"]]

In [11]:
movie.head()

,id,title,keywords,genres,overview,cast,crew
0,19995,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [12]:
movie.isnull().sum()

id          0
title       0
keywords    0
genres      0
overview    3
cast        0
crew        0
dtype: int64

In [13]:
#deleteing if any column is empty
#if present -> overview have NULL values
movie.dropna(inplace =True)

In [14]:
#checking any duplicate is present or not
movie.duplicated().sum() #->0 no duplicate is present

0

In [15]:
movie.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [16]:
#this is not going to work because type(movie.iloc[0].genres) -> string
# list = []
# for dict in movie.iloc[0].genres:
#     list.append(dict["name"])

In [17]:
#need to convert that thing into a list of dict
# import ast
def convert(obj):
    List = []
    for dict in ast.literal_eval(obj):
        List.append(dict["name"])
    return List

In [18]:
#checking the function
L = convert(movie.iloc[0].genres)       
print(L)

['Action', 'Adventure', 'Fantasy', 'Science Fiction']


In [19]:
# movie["genres"].apply(convert) #->run this to see the output
movie["genres"] = movie["genres"].apply(convert)

In [20]:
movie.head() #->genre column has change

,id,title,keywords,genres,overview,cast,crew
0,19995,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[Action, Adventure, Science Fiction]","John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [21]:
#have to do same for keywords
movie["keywords"] = movie["keywords"].apply(convert) 

In [22]:
movie.head()

,id,title,keywords,genres,overview,cast,crew
0,19995,Avatar,"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]","John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [23]:
def convertCast(obj):
    List = []
    for dict in ast.literal_eval(obj):
        List.append(dict["name"])
        if(len(List) == 5):
            break;
            
    return List

In [24]:
l = convertCast(movie.iloc[0].cast)

In [25]:
movie["cast"] = movie["cast"].apply(convertCast)

In [26]:
movie["cast"].values

array([list(['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Stephen Lang', 'Michelle Rodriguez']),
       list(['Johnny Depp', 'Orlando Bloom', 'Keira Knightley', 'Stellan Skarsgård', 'Chow Yun-fat']),
       list(['Daniel Craig', 'Christoph Waltz', 'Léa Seydoux', 'Ralph Fiennes', 'Monica Bellucci']),
       ...,
       list(['Eric Mabius', 'Kristin Booth', 'Crystal Lowe', 'Geoff Gustafson', 'Benjamin Hollingsworth']),
       list(['Daniel Henney', 'Eliza Coupe', 'Bill Paxton', 'Alan Ruck', 'Zhu Shimao']),
       list(['Drew Barrymore', 'Brian Herzlinger', 'Corey Feldman', 'Eric Roberts', 'Griffin Dunne'])],
      dtype=object)

In [27]:
def fetchDirector(obj):
    List = []
    for dict in ast.literal_eval(obj):
        if(dict["job"] == "Director"):
            List.append(dict["name"])
            break
    return List

In [28]:
print(fetchDirector(movie.iloc[0].crew))

['James Cameron']


In [29]:
movie["crew"] = movie["crew"].apply(fetchDirector)

In [30]:
movie.head()

,id,title,keywords,genres,overview,cast,crew
0,19995,Avatar,"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,The Dark Knight Rises,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...,"[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,John Carter,"[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]","John Carter is a war-weary, former military ca...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


In [31]:
#converting the overview sentences into list too.
movie["overview"] = movie["overview"].apply(lambda x : x.split())

In [32]:
movie.head()

,id,title,keywords,genres,overview,cast,crew
0,19995,Avatar,"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]","[Captain, Barbossa,, long, believed, to, be, d...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]","[A, cryptic, message, from, Bond’s, past, send...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,The Dark Knight Rises,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]","[Following, the, death, of, District, Attorney...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,John Carter,"[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]","[John, Carter, is, a, war-weary,, former, mili...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


In [33]:
#removed space between word to stop make confusion
movie["genres"] = movie["genres"].apply(lambda x : [i.replace(" ","") for i in x])
movie["keywords"] = movie["keywords"].apply(lambda x : [i.replace(" ","") for i in x])
movie["cast"] = movie["cast"].apply(lambda x : [i.replace(" ","") for i in x])
movie["crew"] = movie["crew"].apply(lambda x : [i.replace(" ","") for i in x])

In [34]:
movie.head()

,id,title,keywords,genres,overview,cast,crew
0,19995,Avatar,"[cultureclash, future, spacewar, spacecolony, ...","[Action, Adventure, Fantasy, ScienceFiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[ocean, drugabuse, exoticisland, eastindiatrad...","[Adventure, Fantasy, Action]","[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,206647,Spectre,"[spy, basedonnovel, secretagent, sequel, mi6, ...","[Action, Adventure, Crime]","[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,49026,The Dark Knight Rises,"[dccomics, crimefighter, terrorist, secretiden...","[Action, Crime, Drama, Thriller]","[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,49529,John Carter,"[basedonnovel, mars, medallion, spacetravel, p...","[Action, Adventure, ScienceFiction]","[John, Carter, is, a, war-weary,, former, mili...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]


In [35]:
# concatenation of keywords,genres,overview,cast,crew
movie["tag"] = movie["overview"] + movie["genres"] + movie["keywords"] + movie["cast"] + movie["crew"]

In [36]:
new_movie = movie[["id","title","tag"]]

In [37]:
new_movie.head()

,id,title,tag
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [38]:
#need to do stemming to decrease the features
ps = PorterStemmer()
def stem(tag_list):
    list = []
    for tag in tag_list:
        tag = tag.lower() #-> making the word in lower case 
        list.append(ps.stem(tag)) #->stemming the each word
    return " ".join(list) #-> making the sentence from the list becasuse to vectorese that thing

In [39]:
print(stem(new_movie["tag"][0]))

in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav stephenlang michellerodriguez jamescameron


In [40]:
new_movie["tag"] = new_movie["tag"].apply(stem)

C:\Users\priya\AppData\Local\Temp\ipykernel_9668\4102460092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movie["tag"] = new_movie["tag"].apply(stem)


In [41]:
new_movie.head()

,id,title,tag
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [42]:
#need to convert tag into vector to match the similarity because we don't have any way to match them using text
print(new_movie["tag"][0])

in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav stephenlang michellerodriguez jamescameron


In [43]:
#also removing the stop words
cv = CountVectorizer(max_features = 5000, stop_words="english")#->used only first 5000 thousand word order is based on accurancy

In [44]:
vector_tag = cv.fit_transform(new_movie["tag"]).toarray()
vector_tag

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [45]:
cv.vocabulary_ #-> these words are used to make features

{'century': 740,
 'marin': 2827,
 'dispatch': 1296,
 'moon': 3055,
 'uniqu': 4687,
 'mission': 3025,
 'becom': 440,
 'torn': 4562,
 'follow': 1723,
 'order': 3262,
 'protect': 3547,
 'alien': 158,
 'action': 75,
 'adventur': 102,
 'fantasi': 1633,
 'sciencefict': 3946,
 'cultureclash': 1090,
 'futur': 1806,
 'societi': 4164,
 'spacetravel': 4208,
 'futurist': 1808,
 'romanc': 3824,
 'space': 4202,
 'tribe': 4612,
 'alienplanet': 161,
 'soldier': 4171,
 'battl': 423,
 '3d': 44,
 'samworthington': 3907,
 'zoesaldana': 4994,
 'sigourneyweav': 4091,
 'michellerodriguez': 2972,
 'jamescameron': 2316,
 'captain': 678,
 'long': 2719,
 'believ': 453,
 'dead': 1161,
 'ha': 1957,
 'come': 924,
 'life': 2669,
 'head': 2013,
 'edg': 1421,
 'earth': 1409,
 'turner': 4634,
 'elizabeth': 1445,
 'noth': 3203,
 'quit': 3584,
 'ocean': 3224,
 'drugabus': 1371,
 'exoticisland': 1580,
 'loveofone': 2751,
 'slif': 4144,
 'traitor': 4592,
 'shipwreck': 4069,
 'ship': 4068,
 'allianc': 169,
 'afterlif': 114,

In [46]:
len(cv.vocabulary_) #->length of the vocabulary

5000

In [47]:
similarity = cosine_similarity(vector_tag)

In [48]:
similarity

array([[1.        , 0.08006408, 0.08492078, ..., 0.04484485, 0.        ,
        0.        ],
       [0.08006408, 1.        , 0.05892557, ..., 0.023338  , 0.        ,
        0.02512595],
       [0.08492078, 0.05892557, 1.        , ..., 0.02475369, 0.        ,
        0.        ],
       ...,
       [0.04484485, 0.023338  , 0.02475369, ..., 1.        , 0.04174829,
        0.04222003],
       [0.        , 0.        , 0.        , ..., 0.04174829, 1.        ,
        0.08989331],
       [0.        , 0.02512595, 0.        , ..., 0.04222003, 0.08989331,
        1.        ]])

In [49]:
new_movie[new_movie["title"] == "The Dark Knight Rises"].index[0]

3

In [96]:
#function to recommmand top 5 similar movies
def recommended(movie):
    movie_index = new_movie[new_movie["title"] == movie].index[0]
    vector = similarity[movie_index]
    distances = sorted(list(enumerate(vector)),reverse=True,key = lambda x: x[1])[1:6]
    for i in distances:
#         print(i)
        print(new_movie.iloc[i[0]].title)

In [97]:
recommended("The Dark Knight Rises")

The Dark Knight
Batman Begins
Batman Returns
Batman
Batman Forever


In [84]:
import pickle

In [94]:
pickle.dump(new_movie.to_dict(),open("movie_dict.pkl","wb"))

In [95]:
pickle.dump(similarity,open("similarity.pkl","wb"))